In [1]:
from google.colab import drive
drive.mount('/content/drive')
from pathlib import Path
import os
repo_path = Path.cwd()/'drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/notebooks/'
os.chdir(str(repo_path))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# !cp -r /content/drive/MyDrive/calcification-detection-project/calcification_detecion/data_rois.zip /home/
# !unzip /home/data_rois.zip -d /home
# !mv /home/home/vzalevskyi/projects/data_rois /home/data_rois
# !rm -r /home/home

In [3]:
from pathlib import Path
thispath = Path.cwd().resolve()
import yaml

In [4]:
# cfg_path = str(thispath.parent.parent/'deep_learning/config.yml')
# with open(cfg_path, "r") as ymlfile:
#     cfg = yaml.safe_load(ymlfile)

In [5]:
cfg = {
    'model': {
        'activation': 'LeakyReLU',
        'dropout': 0.2,
        'fc_dims': [512, 512],
        'freeze_weights': True,
        'backbone': 'resnet18',
        'pretrained': True
    },
    'dataset': {
        'extract_patches': False,
        'delete_previous': False,
        'extract_patches_method': 'all',
        'patch_size': 224,
        'stride': 100,
        'min_breast_fraction_roi': 0.5,
        'n_jobs': -1,
        'cropped_imgs': True,
        'ignore_diameter_px': 15,
        'patch_images_path': '/home/data_rois/',
        'train_neg_to_pos_ratio': 5,
        'normalization': 'z_score'
    },
    'dataloaders': {
        'train_batch_size': 64,
        'val_batch_size': 256
    },
    'data_aug': {
        'prob': 0
    },
    'training': {
        'criterion': 'BCEWithLogitsLoss',
        'optimizer': 'SGD',
        'optimizer_args': {
            'lr': 0.0001, 'momentum': 0.9
        },
        'lr_scheduler': 'StepLR',
        'lr_scheduler_args': {
            'step_size': 10, 'gamma': 0.1
        },
        'n_epochs': 20,
        'best_metric': 'f1_score',
        'resume_training': False,
        'early_stopping': True,
        'early_stopping_args':{
            'min_diff': 0.0001,
            'max_epoch': 3
        },
        'log_iters': 1000,
        'max_iters_per_epoch': 500
        },
    'experiment_name': 'freezed_resnet18_01'
}

In [6]:
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)

In [7]:
# !pip install transformers

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FocalLoss(nn.CrossEntropyLoss):
    ''' Focal loss for classification tasks on imbalanced datasets '''

    def __init__(self, gamma, alpha=None, ignore_index=-100, reduction='none'):
        super().__init__(weight=alpha, ignore_index=ignore_index, reduction='none')
        self.reduction = reduction
        self.gamma = gamma

    def forward(self, input_, target):
        cross_entropy = super().forward(input_, target)
        # Temporarily mask out ignore index to '0' for valid gather-indices input.
        # This won't contribute final loss as the cross_entropy contribution
        # for these would be zero.
        target = target * (target != self.ignore_index).long()
        input_prob = torch.gather(F.softmax(input_, 1), 1, target.unsqueeze(1))
        loss = torch.pow(1 - input_prob, self.gamma) * cross_entropy
        return torch.mean(loss) if self.reduction == 'mean' \
            else torch.sum(loss) if self.reduction == 'sum' \
            else loss

In [11]:
from pathlib import Path
thispath = Path.cwd().resolve()
import sys; sys.path.insert(0, str(thispath.parent))

from deep_learning.dataset.dataset import INBreast_Dataset_pytorch
from deep_learning.models.base_classifier import CNNClasssifier
import deep_learning.dl_utils as dl_utils

import logging
import torch
import time
import random
import yaml

import numpy as np
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as T

from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
from transformers import SwinForImageClassification

logging.basicConfig(level=logging.INFO)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


def identity_function(arg):
    return arg


def train_model(datasets, dataloaders, data_transforms, model, criterion, optimizer, scheduler, cfg):

    # guarantee reproducibility
    since = time.time()
    random.seed(0)
    torch.manual_seed(1442)
    np.random.seed(0)

    # holders for best model
    best_metric = 0.0
    best_metric_name = cfg['training']['best_metric']

    exp_path = Path.cwd().parent.parent/f'data/deepl_runs/{cfg["experiment_name"]}'
    exp_path.mkdir(exist_ok=True, parents=True)
    best_model_path = exp_path / f'{cfg["experiment_name"]}.pt'
    chkpt_path = exp_path / f'{cfg["experiment_name"]}_chkpt.pt'

    if cfg['training']['resume_training']:
        checkpoint = torch.load(chkpt_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        init_epoch = checkpoint['epoch'] + 1
    else:
        init_epoch = 0

    # tensorboard loggs
    log_dir = exp_path/'tensorboard'
    log_dir.mkdir(exist_ok=True, parents=True)
    writer = SummaryWriter(log_dir=log_dir)

    early_stopping_count = 0
    previous_metric = 0
    previous_loss = 0

    for epoch in range(init_epoch, cfg['training']['n_epochs']):
        logging.info(f'Epoch {epoch+1}/{cfg["training"]["n_epochs"]}')
        logging.info(('-' * 10))

        # resample used negatives to use the large diversity of them that we have
        datasets['train'].update_sample_used(epoch)
        dataloaders['train'] = DataLoader(
            datasets['train'], batch_size=cfg['dataloaders']['train_batch_size'],
            shuffle=True, num_workers=4, drop_last=False)

        for phase in ['train', 'val']:
            # Set model to the corresponding mode and update lr if necessary
            if phase == 'train':
                if epoch != 0:
                    if cfg['training']['lr_scheduler'] == 'ReduceLROnPlateau':
                        scheduler.step(previous_loss)
                    else:
                        scheduler.step()
                writer.add_scalar(f"LearningRate/{phase}", optimizer.param_groups[0]['lr'], epoch)
                model.train()
            else:
                model.eval()

            # define holders for losses, preds and labels
            running_loss = 0.0
            epoch_preds, epoch_labels = [], []

            # Iterate over data.
            if (cfg['training']['max_iters_per_epoch'] is not None) and (phase == 'train'):
                total_its = cfg['training']['max_iters_per_epoch']
            else:
                total_its = len(dataloaders[phase])

            for it, sample in tqdm(enumerate(dataloaders[phase]), total=total_its):
                # Apply transformations and send to device
                sample['img'] = data_transforms[phase](sample['img'])
                inputs = sample['img'].to(device)
                labels = sample['label'].to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward pass (track history if only in train)
                with torch.set_grad_enabled(phase == 'train'):
                    # predict
                    outputs = model(inputs)
                    
                    if 'transformer' in cfg['model']['backbone']:
                        outputs = outputs['logits']
                    # store values
                    epoch_preds.append(np.asarray(
                        torch.sigmoid(outputs.detach()).flatten().cpu()))
                    epoch_labels.append(np.asarray(labels.detach().cpu()))

                    # finish the comp. graph
                    loss = criterion(outputs.flatten(), labels.float())

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # once in a while store the images batch to check
                # if it in [25, 50, 100]:
                #     imgs = T.functional.rgb_to_grayscale(sample['img']).cpu()
                #     writer.add_images(f'Images/{phase}', imgs, epoch)
                #     del imgs

                # get the epoch loss cumulatively
                running_loss += loss.item() * inputs.size(0)

                if phase == 'train':
                    if (it != 0) and ((it % cfg['training']['log_iters']) == 0):
                        # compute and log the metrics for the iteration
                        iter_preds = np.concatenate(epoch_preds)
                        iter_labels = np.concatenate(epoch_labels)
                        iter_loss = running_loss / len(iter_preds)
                        iter_metrics = dl_utils.get_metrics(iter_labels, iter_preds)
                        dl_utils.tensorboard_logs(
                            writer, iter_loss, it+(total_its*epoch), iter_metrics, phase, True)
                    if cfg['training']['max_iters_per_epoch'] is not None:
                        if it == cfg['training']['max_iters_per_epoch']:
                            break

            # compute and log the metrics for the epoch
            epoch_preds = np.concatenate(epoch_preds)
            epoch_labels = np.concatenate(epoch_labels)
            epoch_loss = running_loss / len(epoch_preds)
            metrics = dl_utils.get_metrics(epoch_labels, epoch_preds)
            dl_utils.tensorboard_logs(writer, epoch_loss, epoch, metrics, phase)

            # print status
            epoch_f1 = metrics['f1_score']
            message = f'{phase} Loss: {epoch_loss:.4f} Acc: {metrics["accuracy"]:.4f}' \
                f' F1: {epoch_f1:.4f} AUROC: {metrics["auroc"]:.4f}'
            logging.info(message)

            # save last and best checkpoint
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss}, chkpt_path)

            if phase == 'val':
                if metrics[best_metric_name] > best_metric:
                    best_metric = epoch_f1
                    best_threshold = metrics['threshold']
                    torch.save({
                        'model_state_dict': model.state_dict(),
                        'metrics': metrics,
                        'configuration': cfg
                        }, best_model_path)

                if cfg['training']['early_stopping'] and (epoch != 0):
                #     diff = metrics[best_metric_name] - previous_metric
                #     if diff < cfg['training']['early_stopping_args']['min_diff']:
                #         early_stopping_count += 1
                #     else:
                #         early_stopping_count = 0
                    diff = epoch_loss - previous_loss
                    if -diff < cfg['training']['early_stopping_args']['min_diff']:
                        early_stopping_count += 1
                    else:
                        early_stopping_count = 0
                previous_loss = epoch_loss
                previous_metric = metrics[best_metric_name]

        if cfg['training']['early_stopping']:
            max_epochs = cfg['training']['early_stopping_args']['max_epoch']
            if early_stopping_count == max_epochs:
                msg = f'Early stopping after {max_epochs} epochs without' \
                    f' significant change in val metric'
                logging.info(msg)
                break
        logging.info(('-' * 10))

    time_elapsed = time.time() - since
    message = f'Training complete in {(time_elapsed // 60):.0f}m ' \
        f'{(time_elapsed % 60):.0f}s'
    logging.info(message)
    logging.info(f'Best val {best_metric_name}: {best_metric:4f}, threshold {best_threshold:.4f}')

    # close the tensorboard session
    writer.flush()
    writer.close()

    # load best model weights before returning
    best_model = torch.load(best_model_path)
    model.load_state_dict(best_model['model_state_dict'])
    return model


def main():
    # read the configuration file
    config_path = thispath.parent/"deep_learning/config.yml"
    with open(config_path, "r") as ymlfile:
        cfg = yaml.safe_load(ymlfile)

    # use the configuration for the dataset
    dataset_arguments = cfg['dataset']
    dataset_arguments['patch_images_path'] = Path(dataset_arguments['patch_images_path'])
    datasets = {
        'train': INBreast_Dataset_pytorch(
            partitions=['train'], neg_to_pos_ratio=dataset_arguments['train_neg_to_pos_ratio'],
            balancing_seed=0, **dataset_arguments),
        'val': INBreast_Dataset_pytorch(
            partitions=['validation'], neg_to_pos_ratio=None, **dataset_arguments)
    }

    # use the configuration for the dataloaders
    dataloaders = {
        'val': DataLoader(
            datasets['val'], batch_size=cfg['dataloaders']['val_batch_size'],
            num_workers=4, drop_last=False),
        'train': DataLoader(
            datasets['train'], batch_size=cfg['dataloaders']['train_batch_size'],
            shuffle=True, num_workers=4, drop_last=False)
    }

    # use the configuration for the transformations
    transforms = nn.Sequential(
        T.ColorJitter(brightness=0.4, contrast=0.4, saturation=0, hue=0),
        T.RandomAffine(
            degrees=(0, 20), translate=None, scale=None, shear=(1, 10, 1, 10),
            interpolation=T.InterpolationMode.BILINEAR, fill=0
        ),
        T.RandomPerspective(distortion_scale=0.2),
        T.RandomRotation(degrees=(0, 20)),
        T.RandomRotation(degrees=(90, 110)),
        T.RandomResizedCrop(size=(224, 224), scale=(0.9, 1), ratio=(1, 1)),
        T.RandomAutocontrast(),
        T.RandomHorizontalFlip(),
        T.RandomVerticalFlip()
    )
    transforms = T.RandomApply(transforms=transforms, p=cfg['data_aug']['prob'])
    data_transforms = {
        'train': identity_function if (cfg['data_aug']['prob'] == 0) else transforms,
        'val': identity_function
    }

    # model configs
    if cfg['model']['backbone'] == 'swin_transformer':
        model = SwinForImageClassification.from_pretrained(
            'microsoft/swin-tiny-patch4-window7-224',
            # num_labels=2,
            # id2label={str(0): 'normal', str(1): 'abnormal'},
            # label2id={'normal': str(0), 'abnormal': str(1)},
            num_labels=1,
            # id2label={str(1): 'abnormal'},
            # label2id={'abnormal': str(1)},
            ignore_mismatched_sizes = True,
        )
    else:
        model = CNNClasssifier(
            activation=getattr(nn, cfg['model']['activation'])(),
            dropout=cfg['model']['dropout'],
            fc_dims=cfg['model']['fc_dims'],
            freeze_weights=cfg['model']['freeze_weights'],
            backbone=cfg['model']['backbone'],
            pretrained=cfg['model']['pretrained'],
        )
        model = model.model
    model = model.to(device)


    # training configs
    criterion = getattr(nn, cfg['training']['criterion'])()

    optimizer = getattr(optim, cfg['training']['optimizer'])
    optimizer = optimizer(model.parameters(), **cfg['training']['optimizer_args'])

    scheduler = getattr(lr_scheduler, cfg['training']['lr_scheduler'])
    scheduler = scheduler(optimizer, **cfg['training']['lr_scheduler_args'])

    # train the model
    train_model(datasets, dataloaders, data_transforms, model, criterion, optimizer, scheduler, cfg)


# if __name__ == '__main__':
    # main()


In [14]:
cfg['training']['lr_scheduler'] = 'LinearLR'
cfg['training']['lr_scheduler_args'] = {
  'start_factor': 0.3333333333333333,
  'end_factor': 1.0,
  'total_iters': 15,
  'last_epoch': -1
}
# cfg['training']['lr_scheduler'] = 'CosineAnnealingLR'
# cfg['training']['lr_scheduler_args'] = {
#     'T_max': 7,
#     'eta_min': 5e-6
# }
cfg['training']['optimizer'] = 'AdamW'
cfg['training']['optimizer_args'] = {'lr': 5e-4, 'weight_decay': 0.05}
cfg['training']['normalization'] = 'z_score'
cfg['training']['n_epochs'] = 5
cfg['training']['log_iters'] = 100
cfg['training']['max_iters_per_epoch'] = 300
cfg['dataset']['train_neg_to_pos_ratio'] = 7
cfg['model']['backbone'] = 'swin_transformer'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0
cfg['model']['fc_dims'] = None
cfg['dataloaders']['train_batch_size'] = 32
cfg['data_aug']['prob'] = 0.5
cfg['experiment_name'] = 'swin_transformer_01'

cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
Some weights of SwinForImageClassification were not initialized from the model checkpoint at microsoft/swin-tiny-patch4-window7-224 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([1, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([1]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:root:Epoch 1/5
INFO:root:----------
100%|██████████| 300/300 [02:05<00:00,  2.39it/s]
INFO:root:train Loss: 0.3700 Acc: 0.8157 F1: 0.2051 AUROC: 0.56

In [32]:
from torchvision import models
optimizer = getattr(optim, 'Adam')
model = models.resnet18()
optimizer = optimizer(model.parameters(), lr=0.2)

In [34]:
[group['lr'] for group in optimizer.param_groups]

[0.2]

In [ ]:
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/20
INFO:root:----------
100%|██████████| 410/410 [00:41<00:00,  9.88it/s]
INFO:root:train Loss: 0.5325 Acc: 0.7126 F1: 0.2276 AUROC: 0.5364
100%|██████████| 297/297 [00:31<00:00,  9.57it/s]
INFO:root:val Loss: 0.2793 Acc: 0.6301 F1: 0.1581 AUROC: 0.5992
INFO:root:----------
INFO:root:Epoch 2/20
INFO:root:----------
100%|██████████| 410/410 [00:41<00:00,  9.89it/s]
INFO:root:train Loss: 0.4524 Acc: 0.6749 F1: 0.3322 AUROC: 0.6357
100%|██████████| 297/297 [00:30<00:00,  9.71it/s]
INFO:root:val Loss: 0.2649 Acc: 0.5822 F1: 0.1756 AUROC: 0.6542
INFO:root:----------
INFO:root:Epoch 3/20
INFO:root:----------
100%|██████████| 410/410 [00:41<00:00,  9.82it/s]
INFO:root:train Los

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['experiment_name'] = 'freezed_resnet18_02'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/20
INFO:root:----------
100%|██████████| 410/410 [00:42<00:00,  9.76it/s]
INFO:root:train Loss: 0.4309 Acc: 0.7479 F1: 0.3959 AUROC: 0.6935
100%|██████████| 297/297 [00:30<00:00,  9.58it/s]
INFO:root:val Loss: 0.2536 Acc: 0.7119 F1: 0.2225 AUROC: 0.7178
INFO:root:----------
INFO:root:Epoch 2/20
INFO:root:----------
100%|██████████| 410/410 [00:41<00:00,  9.84it/s]
INFO:root:train Loss: 0.3613 Acc: 0.7300 F1: 0.4530 AUROC: 0.7754
100%|██████████| 297/297 [00:30<00:00,  9.73it/s]
INFO:root:val Loss: 0.2374 Acc: 0.7371 F1: 0.2359 AUROC: 0.7322
INFO:root:----------
INFO:root:Epoch 3/20
INFO:root:----------
100%|██████████| 410/410 [00:41<00:00,  9.89it/s]
INFO:root:train Los

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'min_max'
cfg['experiment_name'] = 'freezed_resnet18_03'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/20
INFO:root:----------
100%|██████████| 410/410 [00:42<00:00,  9.56it/s]
INFO:root:train Loss: 0.4122 Acc: 0.6924 F1: 0.3947 AUROC: 0.7068
100%|██████████| 297/297 [00:30<00:00,  9.70it/s]
INFO:root:val Loss: 0.2574 Acc: 0.7583 F1: 0.2449 AUROC: 0.7230
INFO:root:----------
INFO:root:Epoch 2/20
INFO:root:----------
100%|██████████| 410/410 [00:41<00:00,  9.81it/s]
INFO:root:train Loss: 0.3605 Acc: 0.7906 F1: 0.4810 AUROC: 0.7738
100%|██████████| 297/297 [00:30<00:00,  9.58it/s]
INFO:root:val Loss: 0.2447 Acc: 0.7720 F1: 0.2526 AUROC: 0.7335
INFO:root:----------
INFO:root:Epoch 3/20
INFO:root:----------
100%|██████████| 410/410 [00:42<00:00,  9.73it/s]
INFO:root:train Los

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['dataset']['train_neg_to_pos_ratio'] = 10
cfg['experiment_name'] = 'freezed_resnet18_04'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/20
INFO:root:----------
100%|██████████| 724/724 [01:07<00:00, 10.76it/s]
INFO:root:train Loss: 0.2768 Acc: 0.7942 F1: 0.3203 AUROC: 0.7267
100%|██████████| 297/297 [00:30<00:00,  9.68it/s]
INFO:root:val Loss: 0.2323 Acc: 0.7132 F1: 0.2232 AUROC: 0.7254
INFO:root:----------
INFO:root:Epoch 2/20
INFO:root:----------
100%|██████████| 724/724 [01:07<00:00, 10.79it/s]
INFO:root:train Loss: 0.2443 Acc: 0.7809 F1: 0.3529 AUROC: 0.7856
100%|██████████| 297/297 [00:31<00:00,  9.53it/s]
INFO:root:val Loss: 0.2229 Acc: 0.7537 F1: 0.2430 AUROC: 0.7324
INFO:root:----------
INFO:root:Epoch 3/20
INFO:root:----------
100%|██████████| 724/724 [01:06<00:00, 10.91it/s]
INFO:root:train Los

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.001}
cfg['training']['normalization'] = 'z_score'
cfg['dataset']['train_neg_to_pos_ratio'] = 5
cfg['experiment_name'] = 'freezed_resnet18_05'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/20
INFO:root:----------
100%|██████████| 410/410 [00:47<00:00,  8.67it/s]
INFO:root:train Loss: 0.3965 Acc: 0.7584 F1: 0.4333 AUROC: 0.7385
100%|██████████| 297/297 [00:35<00:00,  8.38it/s]
INFO:root:val Loss: 0.2556 Acc: 0.7654 F1: 0.2543 AUROC: 0.7402
INFO:root:----------
INFO:root:Epoch 2/20
INFO:root:----------
100%|██████████| 410/410 [00:44<00:00,  9.32it/s]
INFO:root:train Loss: 0.3440 Acc: 0.7588 F1: 0.4797 AUROC: 0.7890
100%|██████████| 297/297 [00:31<00:00,  9.53it/s]
INFO:root:val Loss: 0.2172 Acc: 0.7066 F1: 0.2278 AUROC: 0.7366
INFO:root:----------
INFO:root:Epoch 3/20
INFO:root:----------
100%|██████████| 410/410 [00:42<00:00,  9.65it/s]
INFO:root:train Los

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.001}
cfg['training']['normalization'] = 'z_score'
cfg['dataset']['train_neg_to_pos_ratio'] = 5
cfg['dataloaders']['train_batch_size'] = 128
cfg['experiment_name'] = 'freezed_resnet18_06'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/20
INFO:root:----------
100%|██████████| 205/205 [00:50<00:00,  4.08it/s]
INFO:root:train Loss: 0.4221 Acc: 0.7348 F1: 0.4192 AUROC: 0.7235
100%|██████████| 149/149 [00:39<00:00,  3.80it/s]
INFO:root:val Loss: 0.2660 Acc: 0.6982 F1: 0.2291 AUROC: 0.7352
INFO:root:----------
INFO:root:Epoch 2/20
INFO:root:----------
100%|██████████| 205/205 [00:49<00:00,  4.14it/s]
INFO:root:train Loss: 0.3416 Acc: 0.7852 F1: 0.4948 AUROC: 0.7954
100%|██████████| 149/149 [00:39<00:00,  3.81it/s]
INFO:root:val Loss: 0.2285 Acc: 0.7864 F1: 0.2627 AUROC: 0.7433
INFO:root:----------
INFO:root:Epoch 3/20
INFO:root:----------
100%|██████████| 205/205 [00:49<00:00,  4.13it/s]
INFO:root:train Los

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['dataset']['train_neg_to_pos_ratio'] = 5
cfg['model']['backbone'] = 'resnet34'
cfg['dataloaders']['train_batch_size'] = 64
cfg['experiment_name'] = 'freezed_resnet34_01'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/20
INFO:root:----------
100%|██████████| 410/410 [00:57<00:00,  7.08it/s]
INFO:root:train Loss: 0.4023 Acc: 0.7490 F1: 0.4259 AUROC: 0.7315
100%|██████████| 297/297 [00:42<00:00,  6.93it/s]
INFO:root:val Loss: 0.2398 Acc: 0.7802 F1: 0.2526 AUROC: 0.7307
INFO:root:----------
INFO:root:Epoch 2/20
INFO:root:----------
100%|██████████| 410/410 [00:58<00:00,  7.00it/s]
INFO:root:train Loss: 0.3463 Acc: 0.7974 F1: 0.5029 AUROC: 0.7975
100%|██████████| 297/297 [00:42<00:00,  7.03it/s]
INFO:root:val Loss: 0.2331 Acc: 0.7264 F1: 0.2363 AUROC: 0.7368
INFO:root:----------
INFO:root:Epoch 3/20
INFO:root:----------
100%|██████████| 410/410 [00:58<00:00,  7.04it/s]
INFO:root:train Los

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['dataset']['train_neg_to_pos_ratio'] = 5
cfg['model']['backbone'] = 'resnet50'
cfg['dataloaders']['train_batch_size'] = 64
cfg['experiment_name'] = 'freezed_resnet50_01'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/20
INFO:root:----------
100%|██████████| 410/410 [01:34<00:00,  4.32it/s]
INFO:root:train Loss: 0.3532 Acc: 0.7625 F1: 0.4703 AUROC: 0.7798
100%|██████████| 297/297 [01:06<00:00,  4.48it/s]
INFO:root:val Loss: 0.2145 Acc: 0.8058 F1: 0.3014 AUROC: 0.7893
INFO:root:----------
INFO:root:Epoch 2/20
INFO:root:----------
100%|██████████| 410/410 [01:34<00:00,  4.34it/s]
INFO:root:train Loss: 0.3080 Acc: 0.8296 F1: 0.5605 AUROC: 0.8363
100%|██████████| 297/297 [01:06<00:00,  4.49it/s]
INFO:root:val Loss: 0.1967 Acc: 0.7640 F1: 0.2808 AUROC: 0.7967
INFO:root:----------
INFO:root:Epoch 3/20
INFO:root:----------
100%|██████████| 410/410 [01:34<00:00,  4.35it/s]
INFO:root:train Los

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['dataset']['train_neg_to_pos_ratio'] = 5
cfg['model']['backbone'] = 'resnet101'
cfg['dataloaders']['train_batch_size'] = 64
cfg['experiment_name'] = 'freezed_resnet101_01'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


  0%|          | 0.00/171M [00:00<?, ?B/s]

INFO:root:Epoch 1/20
INFO:root:----------
100%|██████████| 410/410 [02:31<00:00,  2.71it/s]
INFO:root:train Loss: 0.3693 Acc: 0.7290 F1: 0.4440 AUROC: 0.7621
100%|██████████| 297/297 [01:44<00:00,  2.84it/s]
INFO:root:val Loss: 0.2345 Acc: 0.7455 F1: 0.2618 AUROC: 0.7678
INFO:root:----------
INFO:root:Epoch 2/20
INFO:root:----------
100%|██████████| 410/410 [02:31<00:00,  2.71it/s]
INFO:root:train Loss: 0.3278 Acc: 0.7960 F1: 0.5187 AUROC: 0.8155
100%|██████████| 297/297 [01:44<00:00,  2.84it/s]
INFO:root:val Loss: 0.2066 Acc: 0.7595 F1: 0.2752 AUROC: 0.7793
INFO:root:----------
INFO:root:Epoch 3/20
INFO:root:----------
100%|██████████| 410/410 [02:31<00:00,  2.71it/s]
INFO:root:train Loss: 0.3127 Acc: 0.8068 F1: 0.5389 AUROC: 0.8347
100%|██████████| 297/297 [01:44<00:00,  2.84it/s]
INFO:root:val Loss: 0.2213 Acc: 0.8231 F1: 0.3068 AUROC: 0.7767
INFO:root:----------
INFO:root:Epoch 4/20
INFO:root:----------
100%|██████████| 410/410 [02:31<00:00,  2.71it/s]
INFO:root:train Loss: 0.3075 

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['dataset']['train_neg_to_pos_ratio'] = 5
cfg['model']['backbone'] = 'resnet152'
cfg['dataloaders']['train_batch_size'] = 64
cfg['experiment_name'] = 'freezed_resnet152_01'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth


  0%|          | 0.00/230M [00:00<?, ?B/s]

INFO:root:Epoch 1/20
INFO:root:----------
100%|██████████| 410/410 [03:27<00:00,  1.97it/s]
INFO:root:train Loss: 0.3531 Acc: 0.7914 F1: 0.4899 AUROC: 0.7833
100%|██████████| 297/297 [02:21<00:00,  2.09it/s]
INFO:root:val Loss: 0.2112 Acc: 0.7962 F1: 0.2929 AUROC: 0.7888
INFO:root:----------
INFO:root:Epoch 2/20
INFO:root:----------
100%|██████████| 410/410 [03:28<00:00,  1.97it/s]
INFO:root:train Loss: 0.3041 Acc: 0.8108 F1: 0.5542 AUROC: 0.8434
100%|██████████| 297/297 [02:22<00:00,  2.09it/s]
INFO:root:val Loss: 0.1962 Acc: 0.7407 F1: 0.2659 AUROC: 0.7956
INFO:root:----------
INFO:root:Epoch 3/20
INFO:root:----------
100%|██████████| 410/410 [03:28<00:00,  1.97it/s]
INFO:root:train Loss: 0.2905 Acc: 0.8320 F1: 0.5814 AUROC: 0.8570
100%|██████████| 297/297 [02:22<00:00,  2.09it/s]
INFO:root:val Loss: 0.2123 Acc: 0.8108 F1: 0.3112 AUROC: 0.7975
INFO:root:----------
INFO:root:Epoch 4/20
INFO:root:----------
100%|██████████| 410/410 [03:27<00:00,  1.97it/s]
INFO:root:train Loss: 0.2847 

In [ ]:
### Unfreezed experiments

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['dataset']['train_neg_to_pos_ratio'] = 5
cfg['model']['backbone'] = 'resnet18'
cfg['model']['freeze_weights'] = False
cfg['dataloaders']['train_batch_size'] = 64
cfg['experiment_name'] = 'resnet18_01'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/20
INFO:root:----------
100%|██████████| 410/410 [01:42<00:00,  4.00it/s]
INFO:root:train Loss: 0.2238 Acc: 0.8742 F1: 0.6884 AUROC: 0.9257
100%|██████████| 297/297 [00:31<00:00,  9.47it/s]
INFO:root:val Loss: 0.0979 Acc: 0.9335 F1: 0.6406 AUROC: 0.9496
INFO:root:----------
INFO:root:Epoch 2/20
INFO:root:----------
100%|██████████| 410/410 [01:41<00:00,  4.04it/s]
INFO:root:train Loss: 0.1085 Acc: 0.9467 F1: 0.8538 AUROC: 0.9819
100%|██████████| 297/297 [00:31<00:00,  9.52it/s]
INFO:root:val Loss: 0.1035 Acc: 0.9419 F1: 0.6734 AUROC: 0.9538
INFO:root:----------
INFO:root:Epoch 3/20
INFO:root:----------
100%|██████████| 410/410 [01:41<00:00,  4.03it/s]
INFO:root:train Los

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['dataset']['train_neg_to_pos_ratio'] = 5
cfg['model']['backbone'] = 'resnet18'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['dataloaders']['train_batch_size'] = 128
cfg['experiment_name'] = 'resnet18_02'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/20
INFO:root:----------
100%|██████████| 205/205 [01:53<00:00,  1.81it/s]
INFO:root:train Loss: 0.3101 Acc: 0.8560 F1: 0.6233 AUROC: 0.8678
100%|██████████| 149/149 [00:40<00:00,  3.69it/s]
INFO:root:val Loss: 0.1075 Acc: 0.9337 F1: 0.6357 AUROC: 0.9495
INFO:root:----------
INFO:root:Epoch 2/20
INFO:root:----------
100%|██████████| 205/205 [01:53<00:00,  1.81it/s]
INFO:root:train Loss: 0.1412 Acc: 0.9332 F1: 0.8176 AUROC: 0.9688
100%|██████████| 149/149 [00:39<00:00,  3.78it/s]
INFO:root:val Loss: 0.1061 Acc: 0.9413 F1: 0.6653 AUROC: 0.9478
INFO:root:----------
INFO:root:Epoch 3/20
INFO:root:----------
100%|██████████| 205/205 [01:53<00:00,  1.81it/s]
INFO:root:train Los

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['dataset']['train_neg_to_pos_ratio'] = 5
cfg['model']['backbone'] = 'resnet18'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.2
cfg['dataloaders']['train_batch_size'] = 128
cfg['experiment_name'] = 'resnet18_03'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/20
INFO:root:----------
100%|██████████| 205/205 [01:53<00:00,  1.81it/s]
INFO:root:train Loss: 0.2607 Acc: 0.8542 F1: 0.6419 AUROC: 0.8981
100%|██████████| 149/149 [00:40<00:00,  3.72it/s]
INFO:root:val Loss: 0.1003 Acc: 0.9327 F1: 0.6331 AUROC: 0.9482
INFO:root:----------
INFO:root:Epoch 2/20
INFO:root:----------
100%|██████████| 205/205 [01:53<00:00,  1.81it/s]
INFO:root:train Loss: 0.1124 Acc: 0.9525 F1: 0.8649 AUROC: 0.9803
100%|██████████| 149/149 [00:40<00:00,  3.66it/s]
INFO:root:val Loss: 0.1354 Acc: 0.9270 F1: 0.6170 AUROC: 0.9484
INFO:root:----------
INFO:root:Epoch 3/20
INFO:root:----------
100%|██████████| 205/205 [01:54<00:00,  1.80it/s]
INFO:root:train Los

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['dataset']['train_neg_to_pos_ratio'] = 10
cfg['model']['backbone'] = 'resnet18'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['dataloaders']['train_batch_size'] = 128
# cfg['data_aug']['prob'] = 0.2
cfg['experiment_name'] = 'resnet18_04'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/20
INFO:root:----------
100%|██████████| 362/362 [03:07<00:00,  1.93it/s]
INFO:root:train Loss: 0.2135 Acc: 0.8755 F1: 0.5287 AUROC: 0.8821
100%|██████████| 149/149 [00:40<00:00,  3.72it/s]
INFO:root:val Loss: 0.1017 Acc: 0.9321 F1: 0.6349 AUROC: 0.9490
INFO:root:----------
INFO:root:Epoch 2/20
INFO:root:----------
100%|██████████| 362/362 [03:07<00:00,  1.93it/s]
INFO:root:train Loss: 0.0884 Acc: 0.9440 F1: 0.7556 AUROC: 0.9745
100%|██████████| 149/149 [00:40<00:00,  3.68it/s]
INFO:root:val Loss: 0.1089 Acc: 0.9361 F1: 0.6485 AUROC: 0.9430
INFO:root:----------
INFO:root:Epoch 3/20
INFO:root:----------
100%|██████████| 362/362 [03:07<00:00,  1.93it/s]
INFO:root:train Los

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['dataset']['train_neg_to_pos_ratio'] = 10
cfg['model']['backbone'] = 'resnet18'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['dataloaders']['train_batch_size'] = 128
cfg['data_aug']['prob'] = 0.3
cfg['experiment_name'] = 'resnet18_04'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/20
INFO:root:----------
100%|██████████| 362/362 [04:31<00:00,  1.33it/s]
INFO:root:train Loss: 0.2605 Acc: 0.8363 F1: 0.4161 AUROC: 0.8036
100%|██████████| 149/149 [00:40<00:00,  3.72it/s]
INFO:root:val Loss: 0.1175 Acc: 0.9091 F1: 0.5525 AUROC: 0.9328
INFO:root:----------
INFO:root:Epoch 2/20
INFO:root:----------
100%|██████████| 362/362 [04:23<00:00,  1.37it/s]
INFO:root:train Loss: 0.1520 Acc: 0.9002 F1: 0.5985 AUROC: 0.9210
100%|██████████| 149/149 [00:39<00:00,  3.73it/s]
INFO:root:val Loss: 0.1173 Acc: 0.9447 F1: 0.6662 AUROC: 0.9391
INFO:root:----------
INFO:root:Epoch 3/20
INFO:root:----------
100%|██████████| 362/362 [04:28<00:00,  1.35it/s]
INFO:root:train Los

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['dataset']['train_neg_to_pos_ratio'] = 10
cfg['model']['backbone'] = 'resnet18'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.2
cfg['model']['fc_dims'] = None
cfg['dataloaders']['train_batch_size'] = 128
cfg['data_aug']['prob'] = 0.2
cfg['experiment_name'] = 'resnet18_06'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()
cfg['model']['fc_dims'] = [512, 512]

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
INFO:root:Epoch 1/20
INFO:root:----------
100%|██████████| 362/362 [02:55<00:00,  2.06it/s]
INFO:root:train Loss: 0.1978 Acc: 0.8302 F1: 0.4521 AUROC: 0.8747
100%|██████████| 149/149 [00:41<00:00,  3.56it/s]
INFO:root:val Loss: 0.1196 Acc: 0.9311 F1: 0.6214 AUROC: 0.9332
INFO:root:----------
INFO:root:Epoch 2/20
INFO:root:----------
100%|██████████| 362/362 

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['dataset']['train_neg_to_pos_ratio'] = 10
cfg['model']['backbone'] = 'resnet34'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['dataloaders']['train_batch_size'] = 64
cfg['data_aug']['prob'] = 0
cfg['experiment_name'] = 'resnet34_01'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/20
INFO:root:----------
100%|██████████| 724/724 [04:22<00:00,  2.76it/s]
INFO:root:train Loss: 0.1609 Acc: 0.8974 F1: 0.6049 AUROC: 0.9231
100%|██████████| 297/297 [00:44<00:00,  6.71it/s]
INFO:root:val Loss: 0.1000 Acc: 0.9375 F1: 0.6600 AUROC: 0.9481
INFO:root:----------
INFO:root:Epoch 2/20
INFO:root:----------
100%|██████████| 724/724 [04:23<00:00,  2.75it/s]
INFO:root:train Loss: 0.0786 Acc: 0.9538 F1: 0.7928 AUROC: 0.9786
100%|██████████| 297/297 [00:43<00:00,  6.85it/s]
INFO:root:val Loss: 0.1192 Acc: 0.9277 F1: 0.6197 AUROC: 0.9419
INFO:root:----------
INFO:root:Epoch 3/20
INFO:root:----------
100%|██████████| 724/724 [04:23<00:00,  2.75it/s]
INFO:root:train Los

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['training']['n_epochs'] = 15
cfg['dataset']['train_neg_to_pos_ratio'] =  20
cfg['model']['backbone'] = 'resnet34'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['dataloaders']['train_batch_size'] = 128
cfg['data_aug']['prob'] = 0.2
cfg['experiment_name'] = 'resnet34_02'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
INFO:root:Epoch 1/15
INFO:root:----------
100%|██████████| 362/362 [05:37<00:00,  1.07it/s]
INFO:root:train Loss: 0.2544 Acc: 0.8575 F1: 0.4624 AUROC: 0.8185
100%|██████████| 149/149 [00:50<00:00,  2.94it/s]
INFO:root:val Loss: 0.1154 Acc: 0.9437 F1: 0.6660 AUROC: 0.9398
INFO:root:----------
INFO:root:Epoch 2/15
INFO:root:----------
100%|██████████| 362/362 

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['training']['n_epochs'] = 15
cfg['dataset']['train_neg_to_pos_ratio'] = 10
cfg['model']['backbone'] = 'resnet50'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['dataloaders']['train_batch_size'] = 128
cfg['data_aug']['prob'] = 0.2
cfg['experiment_name'] = 'resnet50_01'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/15
INFO:root:----------
100%|██████████| 362/362 [09:58<00:00,  1.65s/it]
INFO:root:train Loss: 0.1801 Acc: 0.8802 F1: 0.5425 AUROC: 0.8926
100%|██████████| 149/149 [01:14<00:00,  2.00it/s]
INFO:root:val Loss: 0.1242 Acc: 0.9503 F1: 0.6977 AUROC: 0.9417
INFO:root:----------
INFO:root:Epoch 2/15
INFO:root:----------
100%|██████████| 362/362 [09:50<00:00,  1.63s/it]
INFO:root:train Loss: 0.1130 Acc: 0.9206 F1: 0.6754 AUROC: 0.9576
100%|██████████| 149/149 [01:15<00:00,  1.97it/s]
INFO:root:val Loss: 0.1210 Acc: 0.9395 F1: 0.6540 AUROC: 0.9333
INFO:root:----------
INFO:root:Epoch 3/15
INFO:root:----------
100%|██████████| 362/362 [09:51<00:00,  1.63s/it]
INFO:root:train Los

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['training']['n_epochs'] = 15
cfg['dataset']['train_neg_to_pos_ratio'] = 10
cfg['model']['backbone'] = 'resnet101'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['dataloaders']['train_batch_size'] = 64
cfg['data_aug']['prob'] = 0.2
cfg['experiment_name'] = 'resnet101_01'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/15
INFO:root:----------
100%|██████████| 724/724 [07:34<00:00,  1.59it/s]
INFO:root:train Loss: 0.1769 Acc: 0.8940 F1: 0.5733 AUROC: 0.8980
100%|██████████| 297/297 [01:02<00:00,  4.73it/s]
INFO:root:val Loss: 0.1211 Acc: 0.9127 F1: 0.5715 AUROC: 0.9378
INFO:root:----------
INFO:root:Epoch 2/15
INFO:root:----------
100%|██████████| 724/724 [07:21<00:00,  1.64it/s]
INFO:root:train Loss: 0.1263 Acc: 0.9351 F1: 0.7081 AUROC: 0.9494
100%|██████████| 297/297 [01:01<00:00,  4.79it/s]
INFO:root:val Loss: 0.1039 Acc: 0.9393 F1: 0.6592 AUROC: 0.9469
INFO:root:----------
INFO:root:Epoch 3/15
INFO:root:----------
100%|██████████| 724/724 [07:18<00:00,  1.65it/s]
INFO:root:train Los

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['training']['n_epochs'] = 15
cfg['dataset']['train_neg_to_pos_ratio'] = 10
cfg['model']['backbone'] = 'resnet152'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['model']['fc_dims'] = None
cfg['dataloaders']['train_batch_size'] = 64
cfg['data_aug']['prob'] = 0.2
cfg['experiment_name'] = 'resnet152_01'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/15
INFO:root:----------
100%|██████████| 724/724 [10:03<00:00,  1.20it/s]
INFO:root:train Loss: 0.1572 Acc: 0.8941 F1: 0.5857 AUROC: 0.9203
100%|██████████| 297/297 [01:21<00:00,  3.66it/s]
INFO:root:val Loss: 0.1187 Acc: 0.9257 F1: 0.6108 AUROC: 0.9472
INFO:root:----------
INFO:root:Epoch 2/15
INFO:root:----------
100%|██████████| 724/724 [09:51<00:00,  1.22it/s]
INFO:root:train Loss: 0.1072 Acc: 0.9320 F1: 0.7072 AUROC: 0.9633
100%|██████████| 297/297 [01:21<00:00,  3.64it/s]
INFO:root:val Loss: 0.1142 Acc: 0.9307 F1: 0.6263 AUROC: 0.9483
INFO:root:----------
INFO:root:Epoch 3/15
INFO:root:----------
100%|██████████| 724/724 [09:46<00:00,  1.23it/s]
INFO:root:train Los

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['training']['n_epochs'] = 15
cfg['dataset']['train_neg_to_pos_ratio'] = None
cfg['model']['backbone'] = 'efficientnet_b0'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['model']['fc_dims'] = None
cfg['dataloaders']['train_batch_size'] = 128
cfg['data_aug']['prob'] = 0.2
cfg['experiment_name'] = 'efficientnet_b0_01'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-3dd342df.pth


  0%|          | 0.00/20.5M [00:00<?, ?B/s]

INFO:root:Epoch 1/15
INFO:root:----------
100%|██████████| 362/362 [04:21<00:00,  1.39it/s]
INFO:root:train Loss: 0.2150 Acc: 0.8038 F1: 0.4177 AUROC: 0.8570
100%|██████████| 149/149 [00:40<00:00,  3.72it/s]
INFO:root:val Loss: 0.1116 Acc: 0.9439 F1: 0.6576 AUROC: 0.9252
INFO:root:----------
INFO:root:Epoch 2/15
INFO:root:----------
100%|██████████| 362/362 [04:06<00:00,  1.47it/s]
INFO:root:train Loss: 0.1262 Acc: 0.9285 F1: 0.6843 AUROC: 0.9419
100%|██████████| 149/149 [00:39<00:00,  3.74it/s]
INFO:root:val Loss: 0.1032 Acc: 0.9366 F1: 0.6418 AUROC: 0.9381
INFO:root:----------
INFO:root:Epoch 3/15
INFO:root:----------
100%|██████████| 362/362 [04:04<00:00,  1.48it/s]
INFO:root:train Loss: 0.1083 Acc: 0.9194 F1: 0.6715 AUROC: 0.9597
100%|██████████| 149/149 [00:39<00:00,  3.76it/s]
INFO:root:val Loss: 0.1118 Acc: 0.9124 F1: 0.5688 AUROC: 0.9329
INFO:root:----------
INFO:root:Epoch 4/15
INFO:root:----------
100%|██████████| 362/362 [04:03<00:00,  1.48it/s]
INFO:root:train Loss: 0.0888 

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['training']['n_epochs'] = 15
cfg['dataset']['train_neg_to_pos_ratio'] = None
cfg['model']['backbone'] = 'efficientnet_b1'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['model']['fc_dims'] = None
cfg['dataloaders']['train_batch_size'] = 100
cfg['data_aug']['prob'] = 0.2
cfg['experiment_name'] = 'efficientnet_b1_01'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/15
INFO:root:----------
100%|██████████| 464/464 [05:48<00:00,  1.33it/s]
INFO:root:train Loss: 0.1940 Acc: 0.8338 F1: 0.4654 AUROC: 0.8839
100%|██████████| 190/190 [00:42<00:00,  4.49it/s]
INFO:root:val Loss: 0.1082 Acc: 0.9363 F1: 0.6398 AUROC: 0.9283
INFO:root:----------
INFO:root:Epoch 2/15
INFO:root:----------
100%|██████████| 464/464 [05:22<00:00,  1.44it/s]
INFO:root:train Loss: 0.1196 Acc: 0.9230 F1: 0.6743 AUROC: 0.9491
100%|██████████| 190/190 [00:41<00:00,  4.57it/s]
INFO:root:val Loss: 0.1055 Acc: 0.9416 F1: 0.6646 AUROC: 0.9429
INFO:root:----------
INFO:root:Epoch 3/15
INFO:root:----------
100%|██████████| 464/464 [05:15<00:00,  1.47it/s]
INFO:root:train Los

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['training']['n_epochs'] = 15
cfg['training']['log_iters'] = 64
cfg['dataset']['train_neg_to_pos_ratio'] = 10
cfg['model']['backbone'] = 'efficientnet_b2'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['model']['fc_dims'] = None
cfg['dataloaders']['train_batch_size'] = 100
cfg['data_aug']['prob'] = 0.2
cfg['experiment_name'] = 'efficientnet_b2_01'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/15
INFO:root:----------
100%|██████████| 464/464 [05:40<00:00,  1.36it/s]
INFO:root:train Loss: 0.1890 Acc: 0.8473 F1: 0.4823 AUROC: 0.8848
100%|██████████| 190/190 [00:42<00:00,  4.47it/s]
INFO:root:val Loss: 0.1056 Acc: 0.9284 F1: 0.6196 AUROC: 0.9405
INFO:root:----------
INFO:root:Epoch 2/15
INFO:root:----------
100%|██████████| 464/464 [05:33<00:00,  1.39it/s]
INFO:root:train Loss: 0.1178 Acc: 0.9271 F1: 0.6876 AUROC: 0.9496
100%|██████████| 190/190 [00:42<00:00,  4.47it/s]
INFO:root:val Loss: 0.1051 Acc: 0.9271 F1: 0.6161 AUROC: 0.9355
INFO:root:----------
INFO:root:Epoch 3/15
INFO:root:----------
100%|██████████| 464/464 [05:26<00:00,  1.42it/s]
INFO:root:train Los

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['training']['n_epochs'] = 5
cfg['training']['log_iters'] = 20
cfg['dataset']['train_neg_to_pos_ratio'] = 10
cfg['model']['backbone'] = 'efficientnet_b3'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['model']['fc_dims'] = None
cfg['dataloaders']['train_batch_size'] = 64
cfg['data_aug']['prob'] = 0.2
cfg['experiment_name'] = 'efficientnet_b3_01'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/5
INFO:root:----------
100%|██████████| 724/724 [06:38<00:00,  1.81it/s]
INFO:root:train Loss: 0.1775 Acc: 0.8588 F1: 0.5030 AUROC: 0.8880
100%|██████████| 75/75 [00:57<00:00,  1.31it/s]
INFO:root:val Loss: 0.1090 Acc: 0.9144 F1: 0.5828 AUROC: 0.9470
INFO:root:----------
INFO:root:Epoch 2/5
INFO:root:----------
100%|██████████| 724/724 [06:30<00:00,  1.85it/s]
INFO:root:train Loss: 0.1153 Acc: 0.9295 F1: 0.6935 AUROC: 0.9491
100%|██████████| 75/75 [00:57<00:00,  1.30it/s]
INFO:root:val Loss: 0.0942 Acc: 0.9366 F1: 0.6516 AUROC: 0.9502
INFO:root:----------
INFO:root:Epoch 3/5
INFO:root:----------
100%|██████████| 724/724 [06:28<00:00,  1.86it/s]
INFO:root:train Loss: 0.08

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['training']['n_epochs'] = 5
cfg['training']['log_iters'] = 100
cfg['dataset']['train_neg_to_pos_ratio'] = None
cfg['model']['backbone'] = 'efficientnet_b4'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['model']['fc_dims'] = None
cfg['dataloaders']['train_batch_size'] = 64
cfg['data_aug']['prob'] = 0.2
cfg['experiment_name'] = 'efficientnet_b4_01'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
Downloading: "https://download.pytorch.org/models/efficientnet_b4_rwightman-7eb33cd5.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b4_rwightman-7eb33cd5.pth


  0%|          | 0.00/74.5M [00:00<?, ?B/s]

INFO:root:Epoch 1/5
INFO:root:----------
100%|██████████| 724/724 [08:34<00:00,  1.41it/s]
INFO:root:train Loss: 0.2182 Acc: 0.8361 F1: 0.4298 AUROC: 0.8361
100%|██████████| 75/75 [01:07<00:00,  1.11it/s]
INFO:root:val Loss: 0.1185 Acc: 0.9121 F1: 0.5661 AUROC: 0.9316
INFO:root:----------
INFO:root:Epoch 2/5
INFO:root:----------
100%|██████████| 724/724 [08:24<00:00,  1.43it/s]
INFO:root:train Loss: 0.1347 Acc: 0.9063 F1: 0.6213 AUROC: 0.9329
100%|██████████| 75/75 [01:08<00:00,  1.10it/s]
INFO:root:val Loss: 0.0975 Acc: 0.9407 F1: 0.6631 AUROC: 0.9479
INFO:root:----------
INFO:root:Epoch 3/5
INFO:root:----------
100%|██████████| 724/724 [08:24<00:00,  1.44it/s]
INFO:root:train Loss: 0.1112 Acc: 0.9218 F1: 0.6749 AUROC: 0.9551
100%|██████████| 75/75 [01:07<00:00,  1.11it/s]
INFO:root:val Loss: 0.1052 Acc: 0.9427 F1: 0.6701 AUROC: 0.9451
INFO:root:----------
INFO:root:Epoch 4/5
INFO:root:----------
100%|██████████| 724/724 [08:21<00:00,  1.44it/s]
INFO:root:train Loss: 0.0972 Acc: 0.936

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['training']['n_epochs'] = 15
cfg['training']['log_iters'] = 100
cfg['training']['max_iters_per_epoch'] = 500
cfg['dataset']['train_neg_to_pos_ratio'] = None
cfg['model']['backbone'] = 'efficientnet_b5'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['model']['fc_dims'] = None
cfg['dataloaders']['train_batch_size'] = 32
cfg['data_aug']['prob'] = 0.4
cfg['experiment_name'] = 'efficientnet_b5_01'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/15
INFO:root:----------
100%|██████████| 500/500 [04:37<00:00,  1.80it/s]
INFO:root:train Loss: 0.2346 Acc: 0.8123 F1: 0.3912 AUROC: 0.8097
100%|██████████| 75/75 [01:21<00:00,  1.08s/it]
INFO:root:val Loss: 0.1308 Acc: 0.8878 F1: 0.4938 AUROC: 0.9053
INFO:root:----------
INFO:root:Epoch 2/15
INFO:root:----------
100%|██████████| 500/500 [04:31<00:00,  1.84it/s]
INFO:root:train Loss: 0.1664 Acc: 0.9112 F1: 0.6094 AUROC: 0.8963
100%|██████████| 75/75 [01:20<00:00,  1.08s/it]
INFO:root:val Loss: 0.1245 Acc: 0.9154 F1: 0.5687 AUROC: 0.9139
INFO:root:----------
INFO:root:Epoch 3/15
INFO:root:----------
100%|██████████| 500/500 [04:33<00:00,  1.83it/s]
INFO:root:train Loss: 0

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['training']['n_epochs'] = 15
cfg['training']['log_iters'] = 50
cfg['training']['max_iters_per_epoch'] = 500
cfg['dataset']['train_neg_to_pos_ratio'] = None
cfg['model']['backbone'] = 'efficientnet_b6'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['model']['fc_dims'] = None
cfg['dataloaders']['train_batch_size'] = 32
cfg['data_aug']['prob'] = 0.4
cfg['experiment_name'] = 'efficientnet_b6_01'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
Downloading: "https://download.pytorch.org/models/efficientnet_b6_lukemelas-c76e70fd.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b6_lukemelas-c76e70fd.pth


  0%|          | 0.00/165M [00:00<?, ?B/s]

INFO:root:Epoch 1/15
INFO:root:----------
100%|██████████| 500/500 [05:40<00:00,  1.47it/s]
INFO:root:train Loss: 0.2465 Acc: 0.7628 F1: 0.3553 AUROC: 0.8057
100%|██████████| 75/75 [01:17<00:00,  1.03s/it]
INFO:root:val Loss: 0.1247 Acc: 0.9097 F1: 0.5459 AUROC: 0.9087
INFO:root:----------
INFO:root:Epoch 2/15
INFO:root:----------
100%|██████████| 500/500 [05:33<00:00,  1.50it/s]
INFO:root:train Loss: 0.1658 Acc: 0.9052 F1: 0.5953 AUROC: 0.8967
100%|██████████| 75/75 [01:16<00:00,  1.02s/it]
INFO:root:val Loss: 0.1109 Acc: 0.9409 F1: 0.6504 AUROC: 0.9194
INFO:root:----------
INFO:root:Epoch 3/15
INFO:root:----------
100%|██████████| 500/500 [05:36<00:00,  1.49it/s]
INFO:root:train Loss: 0.1493 Acc: 0.9195 F1: 0.6527 AUROC: 0.9182
100%|██████████| 75/75 [01:16<00:00,  1.02s/it]
INFO:root:val Loss: 0.1077 Acc: 0.9329 F1: 0.6283 AUROC: 0.9337
INFO:root:----------
INFO:root:Epoch 4/15
INFO:root:----------
100%|██████████| 500/500 [05:37<00:00,  1.48it/s]
INFO:root:train Loss: 0.1387 Acc: 0

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['training']['n_epochs'] = 15
cfg['training']['log_iters'] = 50
cfg['training']['max_iters_per_epoch'] = 500
cfg['training']['lr_scheduler_args']['step_size'] = 6
cfg['dataset']['train_neg_to_pos_ratio'] = None
cfg['model']['backbone'] = 'efficientnet_b7'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['model']['fc_dims'] = None
cfg['dataloaders']['train_batch_size'] = 24
cfg['data_aug']['prob'] = 0.4
cfg['experiment_name'] = 'efficientnet_b7_01'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/15
INFO:root:----------
100%|██████████| 500/500 [05:55<00:00,  1.41it/s]
INFO:root:train Loss: 0.2436 Acc: 0.7702 F1: 0.3615 AUROC: 0.8048
100%|██████████| 75/75 [01:45<00:00,  1.40s/it]
INFO:root:val Loss: 0.1577 Acc: 0.8996 F1: 0.5012 AUROC: 0.8707
INFO:root:----------
INFO:root:Epoch 2/15
INFO:root:----------
100%|██████████| 500/500 [05:45<00:00,  1.45it/s]
INFO:root:train Loss: 0.1671 Acc: 0.8941 F1: 0.5747 AUROC: 0.8928
100%|██████████| 75/75 [01:43<00:00,  1.38s/it]
INFO:root:val Loss: 0.1186 Acc: 0.9513 F1: 0.6887 AUROC: 0.9134
INFO:root:----------
INFO:root:Epoch 3/15
INFO:root:----------
100%|██████████| 500/500 [05:48<00:00,  1.44it/s]
INFO:root:train Loss: 0

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['training']['n_epochs'] = 15
cfg['training']['log_iters'] = 50
cfg['training']['max_iters_per_epoch'] = 500
cfg['training']['lr_scheduler_args']['step_size'] = 6
cfg['dataset']['train_neg_to_pos_ratio'] = None
cfg['model']['backbone'] = 'densenet121'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['model']['fc_dims'] = None
cfg['dataloaders']['train_batch_size'] = 32
cfg['data_aug']['prob'] = 0.4
cfg['experiment_name'] = 'densenet121_01'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/15
INFO:root:----------
100%|██████████| 500/500 [02:27<00:00,  3.38it/s]
INFO:root:train Loss: 0.2698 Acc: 0.8204 F1: 0.3957 AUROC: 0.8106
100%|██████████| 75/75 [00:33<00:00,  2.24it/s]
INFO:root:val Loss: 0.1094 Acc: 0.8909 F1: 0.5223 AUROC: 0.9461
INFO:root:----------
INFO:root:Epoch 2/15
INFO:root:----------
100%|██████████| 500/500 [02:18<00:00,  3.62it/s]
INFO:root:train Loss: 0.1769 Acc: 0.9086 F1: 0.6014 AUROC: 0.8918
100%|██████████| 75/75 [00:33<00:00,  2.26it/s]
INFO:root:val Loss: 0.1228 Acc: 0.9314 F1: 0.6185 AUROC: 0.9182
INFO:root:----------
INFO:root:Epoch 3/15
INFO:root:----------
100%|██████████| 500/500 [02:20<00:00,  3.55it/s]
INFO:root:train Loss: 0

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['training']['n_epochs'] = 15
cfg['training']['log_iters'] = 50
cfg['training']['max_iters_per_epoch'] = 500
cfg['training']['lr_scheduler_args']['step_size'] = 6
cfg['dataset']['train_neg_to_pos_ratio'] = None
cfg['model']['backbone'] = 'resnet50'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['model']['fc_dims'] = None
cfg['dataloaders']['train_batch_size'] = 32
cfg['data_aug']['prob'] = 0.4
cfg['experiment_name'] = 'resnet50_02'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

INFO:root:Epoch 1/15
INFO:root:----------
100%|██████████| 500/500 [02:09<00:00,  3.87it/s]
INFO:root:train Loss: 0.2287 Acc: 0.8155 F1: 0.4063 AUROC: 0.8276
100%|██████████| 75/75 [00:31<00:00,  2.36it/s]
INFO:root:val Loss: 0.1393 Acc: 0.9248 F1: 0.5866 AUROC: 0.9169
INFO:root:----------
INFO:root:Epoch 2/15
INFO:root:----------
100%|██████████| 500/500 [02:01<00:00,  4.11it/s]
INFO:root:train Loss: 0.1795 Acc: 0.8967 F1: 0.5754 AUROC: 0.8881
100%|██████████| 75/75 [00:31<00:00,  2.35it/s]
INFO:root:val Loss: 0.1266 Acc: 0.9232 F1: 0.5858 AUROC: 0.9119
INFO:root:----------
INFO:root:Epoch 3/15
INFO:root:----------
100%|██████████| 500/500 [02:04<00:00,  4.01it/s]
INFO:root:train Loss: 0.1660 Acc: 0.8882 F1: 0.5788 AUROC: 0.9075
100%|██████████| 75/75 [00:31<00:00,  2.36it/s]
INFO:root:val Loss: 0.1216 Acc: 0.9117 F1: 0.5619 AUROC: 0.9381
INFO:root:----------
INFO:root:Epoch 4/15
INFO:root:----------
100%|██████████| 500/500 [02:04<00:00,  4.01it/s]
INFO:root:train Loss: 0.1522 Acc: 0

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['training']['n_epochs'] = 20
cfg['training']['log_iters'] = 50
cfg['training']['max_iters_per_epoch'] = 500
cfg['training']['lr_scheduler'] = 'ReduceLROnPlateau'
cfg['training']['lr_scheduler_args'] = {
    'mode': 'min',
    'factor': 0.5,
    'patience': 5,
    'threshold': 0.001,
    'threshold_mode': 'rel', 
    'cooldown': 0,
    'min_lr': 1e-8,
    'eps': 1e-08,
    'verbose': True
}
cfg['training']['early_stopping_args']['max_epoch'] = 10
cfg['dataset']['train_neg_to_pos_ratio'] = None
cfg['model']['backbone'] = 'resnet50'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['model']['fc_dims'] = None
cfg['dataloaders']['train_batch_size'] = 32
cfg['data_aug']['prob'] = 0.4
cfg['experiment_name'] = 'resnet50_03'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/20
INFO:root:----------
100%|██████████| 500/500 [02:08<00:00,  3.90it/s]
INFO:root:train Loss: 0.2214 Acc: 0.8186 F1: 0.4167 AUROC: 0.8302
100%|██████████| 75/75 [00:31<00:00,  2.34it/s]
INFO:root:val Loss: 0.1226 Acc: 0.9103 F1: 0.5521 AUROC: 0.9220
INFO:root:----------
INFO:root:Epoch 2/20
INFO:root:----------
100%|██████████| 500/500 [02:01<00:00,  4.12it/s]
INFO:root:train Loss: 0.1719 Acc: 0.8946 F1: 0.5760 AUROC: 0.9003
100%|██████████| 75/75 [00:32<00:00,  2.33it/s]
INFO:root:val Loss: 0.1259 Acc: 0.9062 F1: 0.5452 AUROC: 0.9281
INFO:root:----------
INFO:root:Epoch 3/20
INFO:root:----------
100%|██████████| 500/500 [02:04<00:00,  4.01it/s]
INFO:root:train Loss: 0

Epoch 00015: reducing learning rate of group 0 to 5.0000e-05.


100%|██████████| 500/500 [02:06<00:00,  3.96it/s]
INFO:root:train Loss: 0.0990 Acc: 0.9261 F1: 0.6812 AUROC: 0.9643
100%|██████████| 75/75 [00:32<00:00,  2.34it/s]
INFO:root:val Loss: 0.1162 Acc: 0.9107 F1: 0.5711 AUROC: 0.9441
INFO:root:----------
INFO:root:Epoch 17/20
INFO:root:----------
100%|██████████| 500/500 [02:06<00:00,  3.95it/s]
INFO:root:train Loss: 0.0970 Acc: 0.9230 F1: 0.6766 AUROC: 0.9643
100%|██████████| 75/75 [00:31<00:00,  2.34it/s]
INFO:root:val Loss: 0.1224 Acc: 0.9187 F1: 0.5907 AUROC: 0.9417
INFO:root:----------
INFO:root:Epoch 18/20
INFO:root:----------
100%|██████████| 500/500 [02:06<00:00,  3.96it/s]
INFO:root:train Loss: 0.0813 Acc: 0.9497 F1: 0.7621 AUROC: 0.9729
100%|██████████| 75/75 [00:32<00:00,  2.34it/s]
INFO:root:val Loss: 0.1199 Acc: 0.9298 F1: 0.6205 AUROC: 0.9377
INFO:root:----------
INFO:root:Epoch 19/20
INFO:root:----------
100%|██████████| 500/500 [02:08<00:00,  3.91it/s]
INFO:root:train Loss: 0.0935 Acc: 0.9277 F1: 0.6922 AUROC: 0.9668
100%|███

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['training']['n_epochs'] = 30
cfg['training']['log_iters'] = 50
cfg['training']['max_iters_per_epoch'] = 400
cfg['training']['lr_scheduler'] = 'ReduceLROnPlateau'
cfg['training']['lr_scheduler_args'] = {
    'mode': 'min',
    'factor': 0.5,
    'patience': 5,
    'threshold': 0.001,
    'threshold_mode': 'rel', 
    'cooldown': 0,
    'min_lr': 1e-8,
    'eps': 1e-08,
    'verbose': True
}
cfg['training']['early_stopping_args']['max_epoch'] = 10
cfg['dataset']['train_neg_to_pos_ratio'] = None
cfg['model']['backbone'] = 'resnet50'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['model']['fc_dims'] = None
cfg['dataloaders']['train_batch_size'] = 32
cfg['data_aug']['prob'] = 0.2
cfg['experiment_name'] = 'resnet50_04'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/30
INFO:root:----------
100%|██████████| 400/400 [01:30<00:00,  4.40it/s]
INFO:root:train Loss: 0.2033 Acc: 0.8341 F1: 0.4483 AUROC: 0.8582
100%|██████████| 75/75 [00:32<00:00,  2.31it/s]
INFO:root:val Loss: 0.1330 Acc: 0.9010 F1: 0.5248 AUROC: 0.9094
INFO:root:----------
INFO:root:Epoch 2/30
INFO:root:----------
100%|██████████| 400/400 [01:24<00:00,  4.72it/s]
INFO:root:train Loss: 0.1593 Acc: 0.9196 F1: 0.6525 AUROC: 0.9219
100%|██████████| 75/75 [00:31<00:00,  2.36it/s]
INFO:root:val Loss: 0.1010 Acc: 0.9439 F1: 0.6789 AUROC: 0.9512
INFO:root:----------
INFO:root:Epoch 3/30
INFO:root:----------
100%|██████████| 400/400 [01:24<00:00,  4.73it/s]
INFO:root:train Loss: 0

Epoch 00008: reducing learning rate of group 0 to 5.0000e-05.


100%|██████████| 400/400 [01:25<00:00,  4.69it/s]
INFO:root:train Loss: 0.0805 Acc: 0.9514 F1: 0.7817 AUROC: 0.9789
100%|██████████| 75/75 [00:31<00:00,  2.35it/s]
INFO:root:val Loss: 0.1297 Acc: 0.9393 F1: 0.6487 AUROC: 0.9419
INFO:root:----------
INFO:root:Epoch 10/30
INFO:root:----------
100%|██████████| 400/400 [01:27<00:00,  4.59it/s]
INFO:root:train Loss: 0.0835 Acc: 0.9465 F1: 0.7595 AUROC: 0.9771
100%|██████████| 75/75 [00:31<00:00,  2.36it/s]
INFO:root:val Loss: 0.1184 Acc: 0.9411 F1: 0.6583 AUROC: 0.9413
INFO:root:----------
INFO:root:Epoch 11/30
INFO:root:----------
100%|██████████| 400/400 [01:24<00:00,  4.73it/s]
INFO:root:train Loss: 0.0704 Acc: 0.9608 F1: 0.8084 AUROC: 0.9832
100%|██████████| 75/75 [00:31<00:00,  2.35it/s]
INFO:root:val Loss: 0.1377 Acc: 0.9443 F1: 0.6673 AUROC: 0.9304
INFO:root:----------
INFO:root:Epoch 12/30
INFO:root:----------
100%|██████████| 400/400 [01:26<00:00,  4.61it/s]
INFO:root:train Loss: 0.0721 Acc: 0.9574 F1: 0.8015 AUROC: 0.9820
100%|███

Epoch 00014: reducing learning rate of group 0 to 2.5000e-05.


100%|██████████| 400/400 [01:28<00:00,  4.54it/s]
INFO:root:train Loss: 0.0636 Acc: 0.9562 F1: 0.7926 AUROC: 0.9856
100%|██████████| 75/75 [00:31<00:00,  2.36it/s]
INFO:root:val Loss: 0.1350 Acc: 0.9104 F1: 0.5412 AUROC: 0.9128
INFO:root:----------
INFO:root:Epoch 16/30
INFO:root:----------
100%|██████████| 400/400 [01:23<00:00,  4.77it/s]
INFO:root:train Loss: 0.0525 Acc: 0.9518 F1: 0.7833 AUROC: 0.9898
100%|██████████| 75/75 [00:31<00:00,  2.35it/s]
INFO:root:val Loss: 0.1630 Acc: 0.9456 F1: 0.6662 AUROC: 0.9301
INFO:root:----------
INFO:root:Epoch 17/30
INFO:root:----------
100%|██████████| 400/400 [01:26<00:00,  4.65it/s]
INFO:root:train Loss: 0.0512 Acc: 0.9560 F1: 0.8008 AUROC: 0.9907
100%|██████████| 75/75 [00:31<00:00,  2.35it/s]
INFO:root:val Loss: 0.1141 Acc: 0.9206 F1: 0.5920 AUROC: 0.9395
INFO:root:----------
INFO:root:Epoch 18/30
INFO:root:----------
100%|██████████| 400/400 [01:27<00:00,  4.57it/s]
INFO:root:train Loss: 0.0576 Acc: 0.9698 F1: 0.8496 AUROC: 0.9872
100%|███

Epoch 00020: reducing learning rate of group 0 to 1.2500e-05.


100%|██████████| 400/400 [01:24<00:00,  4.76it/s]
INFO:root:train Loss: 0.0395 Acc: 0.9767 F1: 0.8828 AUROC: 0.9942
100%|██████████| 75/75 [00:31<00:00,  2.35it/s]
INFO:root:val Loss: 0.1553 Acc: 0.9349 F1: 0.6396 AUROC: 0.9373
INFO:root:----------
INFO:root:Epoch 22/30
INFO:root:----------
100%|██████████| 400/400 [01:25<00:00,  4.68it/s]
INFO:root:train Loss: 0.0405 Acc: 0.9720 F1: 0.8563 AUROC: 0.9926
100%|██████████| 75/75 [00:31<00:00,  2.36it/s]
INFO:root:val Loss: 0.1411 Acc: 0.9273 F1: 0.6170 AUROC: 0.9387
INFO:root:----------
INFO:root:Epoch 23/30
INFO:root:----------
100%|██████████| 400/400 [01:27<00:00,  4.59it/s]
INFO:root:train Loss: 0.0465 Acc: 0.9641 F1: 0.8289 AUROC: 0.9906
100%|██████████| 75/75 [00:31<00:00,  2.36it/s]
INFO:root:val Loss: 0.1421 Acc: 0.9177 F1: 0.5813 AUROC: 0.9324
INFO:root:----------
INFO:root:Epoch 24/30
INFO:root:----------
100%|██████████| 400/400 [01:26<00:00,  4.60it/s]
INFO:root:train Loss: 0.0471 Acc: 0.9550 F1: 0.8022 AUROC: 0.9913
100%|███

Epoch 00026: reducing learning rate of group 0 to 6.2500e-06.


100%|██████████| 400/400 [01:25<00:00,  4.70it/s]
INFO:root:train Loss: 0.0402 Acc: 0.9724 F1: 0.8673 AUROC: 0.9936
100%|██████████| 75/75 [00:31<00:00,  2.36it/s]
INFO:root:val Loss: 0.1416 Acc: 0.9440 F1: 0.6712 AUROC: 0.9401
INFO:root:----------
INFO:root:Epoch 28/30
INFO:root:----------
100%|██████████| 400/400 [01:26<00:00,  4.62it/s]
INFO:root:train Loss: 0.0424 Acc: 0.9747 F1: 0.8721 AUROC: 0.9930
100%|██████████| 75/75 [00:31<00:00,  2.34it/s]
INFO:root:val Loss: 0.1523 Acc: 0.9372 F1: 0.6484 AUROC: 0.9387
INFO:root:----------
INFO:root:Epoch 29/30
INFO:root:----------
100%|██████████| 400/400 [01:25<00:00,  4.65it/s]
INFO:root:train Loss: 0.0349 Acc: 0.9740 F1: 0.8698 AUROC: 0.9948
100%|██████████| 75/75 [00:31<00:00,  2.34it/s]
INFO:root:val Loss: 0.1589 Acc: 0.9254 F1: 0.6087 AUROC: 0.9382
INFO:root:----------
INFO:root:Epoch 30/30
INFO:root:----------
100%|██████████| 400/400 [01:26<00:00,  4.65it/s]
INFO:root:train Loss: 0.0447 Acc: 0.9705 F1: 0.8592 AUROC: 0.9925
100%|███

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['training']['n_epochs'] = 20
cfg['training']['log_iters'] = 50
cfg['training']['max_iters_per_epoch'] = 500
cfg['training']['lr_scheduler'] = 'ReduceLROnPlateau'
cfg['training']['lr_scheduler_args'] = {
    'mode': 'min',
    'factor': 0.5,
    'patience': 5,
    'threshold': 0.001,
    'threshold_mode': 'rel',
    'cooldown': 0,
    'min_lr': 1e-8,
    'eps': 1e-08,
    'verbose': True
}
cfg['training']['early_stopping_args']['max_epoch'] = 10
cfg['dataset']['train_neg_to_pos_ratio'] = None
cfg['model']['backbone'] = 'inception_v3'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0
cfg['model']['fc_dims'] = None
cfg['dataloaders']['train_batch_size'] = 32
cfg['data_aug']['prob'] = 0.4
cfg['experiment_name'] = 'inception_v3_01'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth


  0%|          | 0.00/104M [00:00<?, ?B/s]

INFO:root:Epoch 1/20
INFO:root:----------
  0%|          | 0/500 [00:00<?, ?it/s]


RuntimeError: ignored

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['training']['n_epochs'] = 30
cfg['training']['log_iters'] = 50
cfg['training']['max_iters_per_epoch'] = 400
cfg['training']['lr_scheduler'] = 'ReduceLROnPlateau'
cfg['training']['lr_scheduler_args'] = {
    'mode': 'min',
    'factor': 0.5,
    'patience': 5,
    'threshold': 0.001,
    'threshold_mode': 'rel', 
    'cooldown': 0,
    'min_lr': 1e-8,
    'eps': 1e-08,
    'verbose': True
}
cfg['training']['early_stopping_args']['max_epoch'] = 10
cfg['dataset']['train_neg_to_pos_ratio'] = None
cfg['model']['backbone'] = 'resnet50'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['model']['fc_dims'] = None
cfg['dataloaders']['train_batch_size'] = 128
cfg['data_aug']['prob'] = 0.4
cfg['experiment_name'] = 'resnet50_05'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/30
INFO:root:----------
 90%|█████████ | 362/400 [05:34<00:35,  1.08it/s]
INFO:root:train Loss: 0.1922 Acc: 0.8544 F1: 0.4844 AUROC: 0.8705
100%|██████████| 75/75 [00:32<00:00,  2.34it/s]
INFO:root:val Loss: 0.1747 Acc: 0.9029 F1: 0.5284 AUROC: 0.9113
INFO:root:----------
INFO:root:Epoch 2/30
INFO:root:----------
 90%|█████████ | 362/400 [05:19<00:33,  1.13it/s]
INFO:root:train Loss: 0.1374 Acc: 0.9176 F1: 0.6465 AUROC: 0.9356
100%|██████████| 75/75 [00:32<00:00,  2.33it/s]
INFO:root:val Loss: 0.1032 Acc: 0.9398 F1: 0.6633 AUROC: 0.9552
INFO:root:----------
INFO:root:Epoch 3/30
INFO:root:----------
 90%|█████████ | 362/400 [05:25<00:34,  1.11it/s]
INFO:root:train Loss: 0

Epoch 00008: reducing learning rate of group 0 to 5.0000e-05.


 90%|█████████ | 362/400 [05:37<00:35,  1.07it/s]
INFO:root:train Loss: 0.0849 Acc: 0.9329 F1: 0.7110 AUROC: 0.9723
100%|██████████| 75/75 [00:32<00:00,  2.32it/s]
INFO:root:val Loss: 0.1026 Acc: 0.9318 F1: 0.6340 AUROC: 0.9525
INFO:root:----------
INFO:root:Epoch 10/30
INFO:root:----------
 90%|█████████ | 362/400 [05:27<00:34,  1.11it/s]
INFO:root:train Loss: 0.0724 Acc: 0.9496 F1: 0.7693 AUROC: 0.9798
100%|██████████| 75/75 [00:32<00:00,  2.34it/s]
INFO:root:val Loss: 0.1107 Acc: 0.9185 F1: 0.5918 AUROC: 0.9500
INFO:root:----------
INFO:root:Epoch 11/30
INFO:root:----------
 90%|█████████ | 362/400 [05:38<00:35,  1.07it/s]
INFO:root:train Loss: 0.0784 Acc: 0.9546 F1: 0.7820 AUROC: 0.9766
100%|██████████| 75/75 [00:32<00:00,  2.33it/s]
INFO:root:val Loss: 0.1113 Acc: 0.8905 F1: 0.5167 AUROC: 0.9387
INFO:root:----------
INFO:root:Epoch 12/30
INFO:root:----------
 90%|█████████ | 362/400 [05:30<00:34,  1.09it/s]
INFO:root:train Loss: 0.0734 Acc: 0.9456 F1: 0.7550 AUROC: 0.9788
100%|███

Epoch 00015: reducing learning rate of group 0 to 2.5000e-05.


 90%|█████████ | 362/400 [05:45<00:36,  1.05it/s]
INFO:root:train Loss: 0.0781 Acc: 0.9325 F1: 0.7144 AUROC: 0.9772
100%|██████████| 75/75 [00:32<00:00,  2.33it/s]
INFO:root:val Loss: 0.1098 Acc: 0.9277 F1: 0.6152 AUROC: 0.9472
INFO:root:----------
INFO:root:Epoch 17/30
INFO:root:----------
 90%|█████████ | 362/400 [05:47<00:36,  1.04it/s]
INFO:root:train Loss: 0.0791 Acc: 0.9447 F1: 0.7494 AUROC: 0.9763
100%|██████████| 75/75 [00:32<00:00,  2.31it/s]
INFO:root:val Loss: 0.1485 Acc: 0.9098 F1: 0.5535 AUROC: 0.9341
INFO:root:----------
INFO:root:Epoch 18/30
INFO:root:----------
 90%|█████████ | 362/400 [05:34<00:35,  1.08it/s]
INFO:root:train Loss: 0.0665 Acc: 0.9429 F1: 0.7502 AUROC: 0.9834
100%|██████████| 75/75 [00:32<00:00,  2.32it/s]
INFO:root:val Loss: 0.1051 Acc: 0.9241 F1: 0.6067 AUROC: 0.9479
INFO:root:----------
INFO:root:Epoch 19/30
INFO:root:----------
 90%|█████████ | 362/400 [05:34<00:35,  1.08it/s]
INFO:root:train Loss: 0.0685 Acc: 0.9507 F1: 0.7752 AUROC: 0.9824
100%|███

Epoch 00021: reducing learning rate of group 0 to 1.2500e-05.


 90%|█████████ | 362/400 [05:34<00:35,  1.08it/s]
INFO:root:train Loss: 0.0689 Acc: 0.9527 F1: 0.7812 AUROC: 0.9822
100%|██████████| 75/75 [00:32<00:00,  2.34it/s]
INFO:root:val Loss: 0.1068 Acc: 0.9026 F1: 0.5483 AUROC: 0.9445
INFO:root:----------
INFO:root:Epoch 23/30
INFO:root:----------
 90%|█████████ | 362/400 [05:39<00:35,  1.07it/s]
INFO:root:train Loss: 0.0712 Acc: 0.9393 F1: 0.7375 AUROC: 0.9812
100%|██████████| 75/75 [00:32<00:00,  2.34it/s]
INFO:root:val Loss: 0.1034 Acc: 0.8877 F1: 0.5176 AUROC: 0.9447
INFO:root:----------
INFO:root:Epoch 24/30
INFO:root:----------
 90%|█████████ | 362/400 [05:30<00:34,  1.10it/s]
INFO:root:train Loss: 0.0626 Acc: 0.9560 F1: 0.7947 AUROC: 0.9850
100%|██████████| 75/75 [00:32<00:00,  2.34it/s]
INFO:root:val Loss: 0.1307 Acc: 0.9099 F1: 0.5602 AUROC: 0.9406
INFO:root:----------
INFO:root:Epoch 25/30
INFO:root:----------
 90%|█████████ | 362/400 [05:29<00:34,  1.10it/s]
INFO:root:train Loss: 0.0624 Acc: 0.9564 F1: 0.7970 AUROC: 0.9857
100%|███

Epoch 00027: reducing learning rate of group 0 to 6.2500e-06.


 90%|█████████ | 362/400 [05:48<00:36,  1.04it/s]
INFO:root:train Loss: 0.0711 Acc: 0.9407 F1: 0.7413 AUROC: 0.9815
100%|██████████| 75/75 [00:32<00:00,  2.33it/s]
INFO:root:val Loss: 0.1065 Acc: 0.9240 F1: 0.6119 AUROC: 0.9514
INFO:root:----------
INFO:root:Epoch 29/30
INFO:root:----------
 90%|█████████ | 362/400 [05:29<00:34,  1.10it/s]
INFO:root:train Loss: 0.0592 Acc: 0.9420 F1: 0.7514 AUROC: 0.9867
100%|██████████| 75/75 [00:32<00:00,  2.33it/s]
INFO:root:val Loss: 0.1248 Acc: 0.9528 F1: 0.7092 AUROC: 0.9481
INFO:root:----------
INFO:root:Epoch 30/30
INFO:root:----------
 90%|█████████ | 362/400 [05:36<00:35,  1.08it/s]
INFO:root:train Loss: 0.0681 Acc: 0.9506 F1: 0.7749 AUROC: 0.9828
100%|██████████| 75/75 [00:32<00:00,  2.28it/s]
INFO:root:val Loss: 0.1155 Acc: 0.9213 F1: 0.5938 AUROC: 0.9411
INFO:root:----------
INFO:root:Training complete in 184m 42s
INFO:root:Best val f1_score: 0.709174, threshold 0.0107


In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['training']['n_epochs'] = 15
cfg['training']['log_iters'] = 50
cfg['training']['max_iters_per_epoch'] = 500
cfg['training']['lr_scheduler_args']['step_size'] = 7
cfg['dataset']['train_neg_to_pos_ratio'] = None
cfg['model']['backbone'] = 'efficientnet_b0'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['model']['fc_dims'] = None
cfg['dataloaders']['train_batch_size'] = 128
cfg['data_aug']['prob'] = 0.4
cfg['experiment_name'] = 'efficientnet_b0_02'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/15
INFO:root:----------
 72%|███████▏  | 362/500 [04:48<01:49,  1.26it/s]
INFO:root:train Loss: 0.2302 Acc: 0.7927 F1: 0.3868 AUROC: 0.8254
100%|██████████| 75/75 [00:22<00:00,  3.34it/s]
INFO:root:val Loss: 0.1667 Acc: 0.8688 F1: 0.4264 AUROC: 0.8516
INFO:root:----------
INFO:root:Epoch 2/15
INFO:root:----------
 72%|███████▏  | 362/500 [04:30<01:43,  1.34it/s]
INFO:root:train Loss: 0.1484 Acc: 0.8891 F1: 0.5771 AUROC: 0.9206
100%|██████████| 75/75 [00:22<00:00,  3.36it/s]
INFO:root:val Loss: 0.1112 Acc: 0.9382 F1: 0.6442 AUROC: 0.9348
INFO:root:----------
INFO:root:Epoch 3/15
INFO:root:----------
 72%|███████▏  | 362/500 [04:36<01:45,  1.31it/s]
INFO:root:train Loss: 0

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['training']['n_epochs'] = 15
cfg['training']['log_iters'] = 50
cfg['training']['max_iters_per_epoch'] = 500
cfg['training']['lr_scheduler_args']['step_size'] = 7
cfg['dataset']['train_neg_to_pos_ratio'] = None
cfg['model']['backbone'] = 'efficientnet_b1'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['model']['fc_dims'] = None
cfg['dataloaders']['train_batch_size'] = 64
cfg['data_aug']['prob'] = 0.4
cfg['experiment_name'] = 'efficientnet_b1_02'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
Downloading: "https://download.pytorch.org/models/efficientnet_b1_rwightman-533bc792.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b1_rwightman-533bc792.pth


  0%|          | 0.00/30.1M [00:00<?, ?B/s]

INFO:root:Epoch 1/15
INFO:root:----------
100%|██████████| 500/500 [04:21<00:00,  1.91it/s]
INFO:root:train Loss: 0.2256 Acc: 0.8016 F1: 0.3961 AUROC: 0.8339
100%|██████████| 75/75 [00:26<00:00,  2.88it/s]
INFO:root:val Loss: 0.1276 Acc: 0.9290 F1: 0.5952 AUROC: 0.8968
INFO:root:----------
INFO:root:Epoch 2/15
INFO:root:----------
100%|██████████| 500/500 [04:14<00:00,  1.97it/s]
INFO:root:train Loss: 0.1491 Acc: 0.9049 F1: 0.6112 AUROC: 0.9227
100%|██████████| 75/75 [00:26<00:00,  2.81it/s]
INFO:root:val Loss: 0.1360 Acc: 0.9196 F1: 0.5643 AUROC: 0.8832
INFO:root:----------
INFO:root:Epoch 3/15
INFO:root:----------
100%|██████████| 500/500 [04:16<00:00,  1.95it/s]
INFO:root:train Loss: 0.1425 Acc: 0.8970 F1: 0.5994 AUROC: 0.9280
100%|██████████| 75/75 [00:29<00:00,  2.51it/s]
INFO:root:val Loss: 0.1257 Acc: 0.9307 F1: 0.6030 AUROC: 0.8979
INFO:root:----------
INFO:root:Epoch 4/15
INFO:root:----------
100%|██████████| 500/500 [04:16<00:00,  1.95it/s]
INFO:root:train Loss: 0.1330 Acc: 0

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['training']['n_epochs'] = 15
cfg['training']['log_iters'] = 50
cfg['training']['max_iters_per_epoch'] = 500
cfg['training']['lr_scheduler_args']['step_size'] = 7
cfg['dataset']['train_neg_to_pos_ratio'] = None
cfg['model']['backbone'] = 'efficientnet_b2'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['model']['fc_dims'] = None
cfg['dataloaders']['train_batch_size'] = 64
cfg['data_aug']['prob'] = 0.4
cfg['experiment_name'] = 'efficientnet_b2_02'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
Downloading: "https://download.pytorch.org/models/efficientnet_b2_rwightman-bcdf34b7.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b2_rwightman-bcdf34b7.pth


  0%|          | 0.00/35.2M [00:00<?, ?B/s]

INFO:root:Epoch 1/15
INFO:root:----------
100%|██████████| 500/500 [04:32<00:00,  1.84it/s]
INFO:root:train Loss: 0.2152 Acc: 0.8135 F1: 0.4094 AUROC: 0.8394
100%|██████████| 75/75 [00:27<00:00,  2.73it/s]
INFO:root:val Loss: 0.1380 Acc: 0.8960 F1: 0.5142 AUROC: 0.9171
INFO:root:----------
INFO:root:Epoch 2/15
INFO:root:----------
100%|██████████| 500/500 [04:18<00:00,  1.93it/s]
INFO:root:train Loss: 0.1506 Acc: 0.9143 F1: 0.6280 AUROC: 0.9179
100%|██████████| 75/75 [00:26<00:00,  2.80it/s]
INFO:root:val Loss: 0.1433 Acc: 0.8903 F1: 0.4916 AUROC: 0.8978
INFO:root:----------
INFO:root:Epoch 3/15
INFO:root:----------
100%|██████████| 500/500 [04:24<00:00,  1.89it/s]
INFO:root:train Loss: 0.1392 Acc: 0.8985 F1: 0.6062 AUROC: 0.9303
100%|██████████| 75/75 [00:27<00:00,  2.72it/s]
INFO:root:val Loss: 0.2545 Acc: 0.8622 F1: 0.4122 AUROC: 0.8652
INFO:root:----------
INFO:root:Epoch 4/15
INFO:root:----------
100%|██████████| 500/500 [04:24<00:00,  1.89it/s]
INFO:root:train Loss: 0.1326 Acc: 0

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['training']['n_epochs'] = 15
cfg['training']['log_iters'] = 50
cfg['training']['max_iters_per_epoch'] = 500
cfg['training']['lr_scheduler_args']['step_size'] = 7
cfg['dataset']['train_neg_to_pos_ratio'] = None
cfg['model']['backbone'] = 'efficientnet_b3'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['model']['fc_dims'] = None
cfg['dataloaders']['train_batch_size'] = 64
cfg['data_aug']['prob'] = 0.4
cfg['experiment_name'] = 'efficientnet_b3_02'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
INFO:root:Epoch 1/15
INFO:root:----------
100%|██████████| 500/500 [05:04<00:00,  1.64it/s]
INFO:root:train Loss: 0.2151 Acc: 0.8519 F1: 0.4502 AUROC: 0.8366
100%|██████████| 75/75 [00:34<00:00,  2.17it/s]
INFO:root:val Loss: 0.1085 Acc: 0.9325 F1: 0.6309 AUROC: 0.9448
INFO:root:----------
INFO:root:Epoch 2/15
INFO:root:----------
100%|██████████| 500/500 [04:50<00:00,  1.72it/s]
INFO:root:train Loss: 0.1471 Acc: 0.9004 F1: 0.6024 AUROC: 0.9209
100%|██████████| 75/75 [00:34<00:00,  2.19it/s]
INFO:root:val Loss: 0.1005 Acc: 0.9420 F1: 0.6675 AUROC: 0.9510
INFO:root:----------
INFO:root:Epoch 3/15
INFO:root:----------
100%|██████████| 500/500 [04:55<00:00,  1.69it/s]
INFO:root:train Loss: 0

In [ ]:
cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['normalization'] = 'z_score'
cfg['training']['n_epochs'] = 15
cfg['training']['log_iters'] = 25
cfg['training']['max_iters_per_epoch'] = 100
cfg['training']['lr_scheduler_args']['step_size'] = 7
cfg['dataset']['train_neg_to_pos_ratio'] = None
cfg['model']['backbone'] = 'efficientnet_b0'
cfg['model']['freeze_weights'] = False
cfg['model']['dropout'] = 0.5
cfg['model']['fc_dims'] = None
cfg['dataloaders']['train_batch_size'] = 128
cfg['data_aug']['prob'] = 0.4
cfg['experiment_name'] = 'efficientnet_b0_03'
cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det/deep_learning/dataset/dataset.py:46: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=False, max_lesion_diam_mm=None, use_muscle_mask=False
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-3dd342df.pth


  0%|          | 0.00/20.5M [00:00<?, ?B/s]

INFO:root:Epoch 1/15
INFO:root:----------
100%|██████████| 100/100 [01:23<00:00,  1.20it/s]
INFO:root:train Loss: 0.3596 Acc: 0.5471 F1: 0.2209 AUROC: 0.6464
100%|██████████| 75/75 [00:21<00:00,  3.52it/s]
INFO:root:val Loss: 0.2417 Acc: 0.6376 F1: 0.1937 AUROC: 0.6867
INFO:root:----------
INFO:root:Epoch 2/15
INFO:root:----------
100%|██████████| 100/100 [01:21<00:00,  1.22it/s]
INFO:root:train Loss: 0.2174 Acc: 0.8736 F1: 0.4910 AUROC: 0.8213
100%|██████████| 75/75 [00:22<00:00,  3.38it/s]
INFO:root:val Loss: 0.1502 Acc: 0.8787 F1: 0.4394 AUROC: 0.8640
INFO:root:----------
INFO:root:Epoch 3/15
INFO:root:----------
100%|██████████| 100/100 [01:15<00:00,  1.33it/s]
INFO:root:train Loss: 0.1684 Acc: 0.8903 F1: 0.5607 AUROC: 0.8990
100%|██████████| 75/75 [00:27<00:00,  2.73it/s]
INFO:root:val Loss: 0.1195 Acc: 0.9243 F1: 0.5808 AUROC: 0.9149
INFO:root:----------
INFO:root:Epoch 4/15
INFO:root:----------
100%|██████████| 100/100 [01:23<00:00,  1.19it/s]
INFO:root:train Loss: 0.1734 Acc: 0

In [ ]:

# # _C.TRAIN = CN()
# # _C.TRAIN.START_EPOCH = 0
# # _C.TRAIN.EPOCHS = 300
# # _C.TRAIN.WARMUP_EPOCHS = 20
# # _C.TRAIN.WEIGHT_DECAY = 0.05
# # _C.TRAIN.BASE_LR = 5e-4
# # _C.TRAIN.WARMUP_LR = 5e-7
# # _C.TRAIN.MIN_LR = 5e-6
# # Clip gradient norm
# # _C.TRAIN.CLIP_GRAD = 5.0

# # LR scheduler
# _C.TRAIN.LR_SCHEDULER = CN()
# _C.TRAIN.LR_SCHEDULER.NAME = 'cosine'
# # Epoch interval to decay LR, used in StepLRScheduler
# _C.TRAIN.LR_SCHEDULER.DECAY_EPOCHS = 30
# # LR decay rate, used in StepLRScheduler
# _C.TRAIN.LR_SCHEDULER.DECAY_RATE = 0.1

# # Optimizer
# _C.TRAIN.OPTIMIZER = CN()
# _C.TRAIN.OPTIMIZER.NAME = 'adamw'
# # Optimizer Epsilon
# _C.TRAIN.OPTIMIZER.EPS = 1e-8
# # Optimizer Betas
# _C.TRAIN.OPTIMIZER.BETAS = (0.9, 0.999)
# # SGD momentum
# _C.TRAIN.OPTIMIZER.MOMENTUM = 0.9